### Tasks
Part 1: Upload the data

Part 2: Analyze your data and create a plan for data preparation

Part 3: Data cleansing (missing values, outliers, duplicates, data consistently)

Part 4: Encode categorical data

Part 5: Upload the deliverables to GitHub

In [235]:
import pandas as pd
import numpy as np
df = pd.read_csv('data_cleaning.csv',low_memory=False)

### Data Inspection

In [236]:
df.shape

(30, 7)

In [237]:
# conclusion: some null data, birthyear type is float (weird)
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TransactionID  30 non-null     int64  
 1   ClientID       30 non-null     int64  
 2   BirthYear      28 non-null     float64
 3   Amount         30 non-null     int64  
 4   Profession     28 non-null     object 
 5   Department     30 non-null     int64  
 6   Risk           30 non-null     object 
dtypes: float64(1), int64(4), object(2)
memory usage: 1.8+ KB


In [238]:
df.head(10)
#we can observe case sensitivity

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
0,4,34985,1923.0,5670,manager,78,Low
1,16,34997,1923.0,2399090,developer,78,High
2,25,35006,1923.0,33050,HR,78,High
3,12,34993,1939.0,23430,professor,78,Low
4,21,35002,1939.0,16770,manager,78,Low
5,11,34992,1944.0,21210,researcher,78,Medium
6,20,35001,1944.0,14550,student,78,Medium
7,3,34984,1945.0,3450,student,78,Medium
8,19,35000,1949.0,12330,barmen,78,High
9,9,34990,1953.0,16770,Manager,78,Medium


In [239]:
# show unique values of a column
# Need harmonization within the labels:
# 'manager', 'Manager'
# 'student','edudient','Student'
# 'bdm','BDM'
df["Profession"].unique()

array(['manager', 'developer', 'HR', 'professor', 'researcher', 'student',
       'barmen', 'Manager', 'bdm', nan, 'hr', 'etudient', 'BDM',
       'Hairdresser', 'Student', 'Driver', 'sailer'], dtype=object)

### CLEANING ACTION PLAN 

1. Deal with NaN values
2. Formatting BirthYear type into integer
3. Clean Profession labels 
4. Check consistency & duplicates
5. Remove outliers

### General Formatting 

In [240]:
#lowering string to lowercase to deal with BDM bdm etc.
df['Profession'] = df['Profession'].astype('str')
df['Risk'] = df['Risk'].astype('str')
df['Profession'] = df['Profession'].str.lower()
df['Risk'] = df['Risk'].str.lower()

In [241]:
df["Profession"].unique()

array(['manager', 'developer', 'hr', 'professor', 'researcher', 'student',
       'barmen', 'bdm', 'nan', 'etudient', 'hairdresser', 'driver',
       'sailer'], dtype=object)

In [242]:
df['Profession'] = df['Profession'].replace('etudient','student')

In [243]:
df["Profession"].unique()

array(['manager', 'developer', 'hr', 'professor', 'researcher', 'student',
       'barmen', 'bdm', 'nan', 'hairdresser', 'driver', 'sailer'],
      dtype=object)

### Dealing with Nan values

In [244]:
# Deal with NaN values
null_displ = df[(df['BirthYear'].isnull()==True) | (df['Profession']=='nan')]
null_displ

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
15,28,35008,1967.0,46370,nan,78,high
16,29,35008,1976.0,50810,nan,78,medium
28,22,34987,NaN,18990,sailer,78,high
29,7,34988,NaN,12330,manager,78,medium


In [245]:
# Let's replace NaN year value by 0 and NaN profession value by unknown
df['BirthYear'].fillna(0,inplace=True)

In [246]:
df['BirthYear'] = df['BirthYear'].astype('int')
df['BirthYear'].dtype

dtype('int32')

In [247]:
pivot = df.groupby('ClientID').agg({'TransactionID':'count'}).sort_values(by='TransactionID',ascending=False)
pivot.head(6)

,TransactionID
ClientID,
35008,3
34987,3
34988,2
34989,2
34991,2
34997,1


In [248]:
sus_clients = pivot.head(5).index

In [249]:
df.loc[df['ClientID'].isin(sus_clients)].sort_values(by="ClientID",ascending=False)

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
14,27,35008,1967,41930,bdm,78,low
15,28,35008,1967,46370,nan,78,high
16,29,35008,1976,50810,nan,78,medium
19,10,34991,1988,18990,bdm,78,low
20,30,34991,1988,55250,bdm,78,high
10,8,34989,1958,14550,hr,78,high
13,24,34989,1967,27870,hr,78,medium
22,23,34988,1999,25650,manager,78,low
29,7,34988,0,12330,manager,78,medium
11,6,34987,1967,10110,manager,78,medium


In [250]:
# changing values for suspicious clients
# df.loc[index].at['column']

In [251]:
# CASE ID 34987, input birthyear and correctjob
df.at[28,'BirthYear'] = 1967
df.at[28,'Profession'] = 'manager'

In [252]:
# CASE ID 34988, input birthyear
df.at[29,'BirthYear'] = 1999

In [253]:
# CASE ID 34989, input birthyear taking 1958 BECAUSE first transaction precedence (assumption)
df.at[13,'BirthYear'] = 1958

In [254]:
# CASE ID 35008, input birthyear and correct job
df.at[16,'BirthYear'] = 1967
df.at[16,'Profession'] = 'bdm'
df.at[15,'Profession'] = 'bdm'

In [255]:
#checking
df.loc[df['ClientID'].isin(sus_clients)].sort_values(by="ClientID",ascending=False)

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
14,27,35008,1967,41930,bdm,78,low
15,28,35008,1967,46370,bdm,78,high
16,29,35008,1967,50810,bdm,78,medium
19,10,34991,1988,18990,bdm,78,low
20,30,34991,1988,55250,bdm,78,high
10,8,34989,1958,14550,hr,78,high
13,24,34989,1958,27870,hr,78,medium
22,23,34988,1999,25650,manager,78,low
29,7,34988,1999,12330,manager,78,medium
11,6,34987,1967,10110,manager,78,medium


In [256]:
#checking the rest 
df.loc[~df['ClientID'].isin(sus_clients)].sort_values(by="BirthYear",ascending=False)

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
27,13,34994,2017,25650,driver,78,medium
26,2,34983,2015,1230,barmen,78,high
25,1,34982,2013,12900,student,78,low
24,26,35007,1999,37490,student,78,medium
23,14,34995,1999,27870,hairdresser,78,low
21,18,34999,1988,10110,student,78,low
17,5,34986,1978,7890,hr,78,high
18,17,34998,1978,7890,student,78,medium
9,9,34990,1953,16770,manager,78,medium
8,19,35000,1949,12330,barmen,78,high


In [257]:
df[ (df['Profession'] != 'student') & (df['BirthYear'] > 2004) ]

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
26,2,34983,2015,1230,barmen,78,high
27,13,34994,2017,25650,driver,78,medium


In [258]:
# taking the mod
df['BirthYear'].value_counts()

1967    6
1999    4
1923    3
1988    3
1939    2
1944    2
1958    2
1978    2
1945    1
1949    1
1953    1
2013    1
2015    1
2017    1
Name: BirthYear, dtype: int64

In [259]:
# CHANGING YOUNG BABIES TO MODE
df.at[26,'BirthYear'] = 1967
df.at[27,'BirthYear'] = 1967

In [260]:
# quick check
df

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
0,4,34985,1923,5670,manager,78,low
1,16,34997,1923,2399090,developer,78,high
2,25,35006,1923,33050,hr,78,high
3,12,34993,1939,23430,professor,78,low
4,21,35002,1939,16770,manager,78,low
5,11,34992,1944,21210,researcher,78,medium
6,20,35001,1944,14550,student,78,medium
7,3,34984,1945,3450,student,78,medium
8,19,35000,1949,12330,barmen,78,high
9,9,34990,1953,16770,manager,78,medium


In [261]:
stats = df.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
TransactionID,30.0,15.500000,8.803408,1.0,8.25,15.5,22.75,30.0,14.5
ClientID,30.0,34993.833333,8.132876,34982.0,34987.25,34991.5,34999.75,35008.0,12.5
BirthYear,30.0,1965.433333,24.426785,1923.0,1946.00,1967.0,1985.50,2013.0,39.5
Amount,30.0,101009.666667,434261.568869,1230.0,12330.00,18990.0,29535.00,2399090.0,17205.0
Department,30.0,78.000000,0.000000,78.0,78.00,78.0,78.00,78.0,0.0


In [262]:
outliers = pd.DataFrame(columns=df.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = df[(df[col] < lower) | 
                   (df[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)

C:\Users\monkeyluffy14\AppData\Local\Temp\ipykernel_27208\1561756670.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers = outliers.append(results)
C:\Users\monkeyluffy14\AppData\Local\Temp\ipykernel_27208\1561756670.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers = outliers.append(results)
C:\Users\monkeyluffy14\AppData\Local\Temp\ipykernel_27208\1561756670.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers = outliers.append(results)
C:\Users\monkeyluffy14\AppData\Local\Temp\ipykernel_27208\1561756670.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers = outliers.append(results)
C:\Users\mon

In [263]:
outliers

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk,Outlier
1,16,34997,1923,2399090,developer,78,high,Amount


In [264]:
df

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
0,4,34985,1923,5670,manager,78,low
1,16,34997,1923,2399090,developer,78,high
2,25,35006,1923,33050,hr,78,high
3,12,34993,1939,23430,professor,78,low
4,21,35002,1939,16770,manager,78,low
5,11,34992,1944,21210,researcher,78,medium
6,20,35001,1944,14550,student,78,medium
7,3,34984,1945,3450,student,78,medium
8,19,35000,1949,12330,barmen,78,high
9,9,34990,1953,16770,manager,78,medium


In [265]:
#removing outliers
df.drop(labels=1, axis=0,inplace=True)

In [266]:
df

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
0,4,34985,1923,5670,manager,78,low
2,25,35006,1923,33050,hr,78,high
3,12,34993,1939,23430,professor,78,low
4,21,35002,1939,16770,manager,78,low
5,11,34992,1944,21210,researcher,78,medium
6,20,35001,1944,14550,student,78,medium
7,3,34984,1945,3450,student,78,medium
8,19,35000,1949,12330,barmen,78,high
9,9,34990,1953,16770,manager,78,medium
10,8,34989,1958,14550,hr,78,high


### ENCODING

In [267]:
df = pd.get_dummies(df, columns=["Profession"])
scale_mapper = {"low":1, "medium":2, "high":3}
df["Risk"] = df["Risk"].replace(scale_mapper)

In [268]:
df

,TransactionID,ClientID,BirthYear,Amount,Department,Risk,Profession_barmen,Profession_bdm,Profession_driver,Profession_hairdresser,Profession_hr,Profession_manager,Profession_professor,Profession_researcher,Profession_student
0,4,34985,1923,5670,78,1,0,0,0,0,0,1,0,0,0
2,25,35006,1923,33050,78,3,0,0,0,0,1,0,0,0,0
3,12,34993,1939,23430,78,1,0,0,0,0,0,0,1,0,0
4,21,35002,1939,16770,78,1,0,0,0,0,0,1,0,0,0
5,11,34992,1944,21210,78,2,0,0,0,0,0,0,0,1,0
6,20,35001,1944,14550,78,2,0,0,0,0,0,0,0,0,1
7,3,34984,1945,3450,78,2,0,0,0,0,0,0,0,0,1
8,19,35000,1949,12330,78,3,1,0,0,0,0,0,0,0,0
9,9,34990,1953,16770,78,2,0,0,0,0,0,1,0,0,0
10,8,34989,1958,14550,78,3,0,0,0,0,1,0,0,0,0


In [270]:
df.to_csv('cleaned_dataset.csv', index=False)